In [19]:
import os
import nest_asyncio
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langsmith import Client
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.smith import RunEvalConfig, run_on_dataset
import bs4
from docx import Document as DocxDocument
from langchain import hub
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import json
import time
from sklearn.metrics import accuracy_score, f1_score

# To Avoid the Error on Jupyter Notebook (RuntimeError: This Event Loop Is Already Running)
# Patch Asyncio To Allow Nested Event Loops

nest_asyncio.apply()

In [2]:
load_dotenv(find_dotenv())
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [3]:
# Function to load a .docx document
def load_docx_document(file_path):
    doc = DocxDocument(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [4]:
# Set the path to the specific file
# Assuming the script/notebook is in the "notebooks" directory and "data" is a sibling directory
current_working_directory = os.getcwd()
parent_directory = os.path.dirname(current_working_directory)
file_path = os.path.join(parent_directory, "data", "Robinson Advisory.docx")

# Load the .docx Document
docx_text = load_docx_document(file_path)

# Convert loaded text to LangChain document format
docs = [Document(page_content=docx_text)]

In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [6]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [8]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is the termination notice? "
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/home/henock/Desktop/10_Academy/week_11/High-Precision-Legal-Expert-LLM/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


6

In [9]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

6

In [14]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0, model_name="gpt-4")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

response = final_rag_chain.invoke({"question": question})
print(response)

The termination notice is fourteen (14) days' prior written notice. Either party may terminate the agreement for any reason, with or without cause. However, the Company may terminate the agreement immediately and without prior notice if the Advisor refuses or is unable to perform the Services, or is in breach of any provision of the Agreement.


In [15]:
# Load evaluation data from JSON
def load_evaluation_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

evaluation_data = load_evaluation_data('evaluation_data.json')

In [16]:
# Function to get RAG response and measure response time
def get_rag_response(question):
    start_time = time.time()
    response = final_rag_chain.invoke({"question": question})
    end_time = time.time()
    response_time = end_time - start_time
    return response, response_time

In [17]:
# Function to run automated tests
def run_automated_tests(evaluation_data):
    results = []
    for contract in evaluation_data['contracts']:
        for qa in contract['questions']:
            predicted_answer, response_time = get_rag_response(qa['question'])
            results.append({
                "question": qa['question'],
                "predicted_answer": predicted_answer,
                "true_answer": qa['answer'],
                "response_time": response_time
            })
    return results

In [20]:
# Function to evaluate accuracy and relevance
def evaluate_results(results):
    true_answers = [result['true_answer'] for result in results]
    predicted_answers = [result['predicted_answer'] for result in results]
    
    accuracy = accuracy_score(true_answers, predicted_answers)
    relevance = f1_score(true_answers, predicted_answers, average='weighted')
    
    report = {
        "accuracy": accuracy,
        "relevance": relevance,
        "details": results
    }
    return report

In [21]:
# Execute the evaluation pipeline
results = run_automated_tests(evaluation_data)
report = evaluate_results(results)

In [22]:
# Print the evaluation report
print(json.dumps(report, indent=4))

{
    "accuracy": 0.0,
    "relevance": 0.0,
    "details": [
        {
            "question": "Who are the parties to the Agreement and what are their defined names?",
            "predicted_answer": "The parties to the Agreement are Cloud Investments Ltd. and an individual named Jack Robinson. Their defined names in the Agreement are \"Company\" for Cloud Investments Ltd. and \"Advisor\" for Jack Robinson.",
            "true_answer": "Cloud Investments Ltd. (\u201cCompany\u201d) and Jack Robinson (\u201cAdvisor\u201d)",
            "response_time": 6.1324543952941895
        },
        {
            "question": "What is the termination notice?",
            "predicted_answer": "The termination notice is fourteen (14) days' prior written notice. However, the Company may terminate the Agreement immediately and without prior notice if the Advisor refuses or is unable to perform the Services, or is in breach of any provision of this Agreement.",
            "true_answer": "According to